<h1> Лабороторна робота N1
<h3>Частина 2</h3>
Необхідно виконати завдання, використовуючи  pandas dataframe, проаналізувати часові витрати на виконання процедур (профілювання часу виконання здійснити за допомогою модуля timeit).
<ul><li>Звантажити та відкрити (вручну або через запропонований скрипт на сайті) наступний датасет: Individual Household Electric Power Consumption Dataset 
</li>
<li>Здійснити data cleaning</li>
<li>Окремими функціями сформувати вибірки:<ul><li>Обрати всі записи, у яких загальна активна споживана потужність перевищує 5 кВт.</li>
    <li>Обрати всі записи, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.
</li>
    <li>Обрати випадковим чином 500000 записів (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії
</li>
    <li>Обрати ті записи, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.</li>
    </ul>
</li>
<li>Пронормувати та стандартизувати вибраний датасет
</li>
<li>Підрахувати коефіцієнт Пірсона та Спірмена для двох integer/real атрибутів.</li>
<li>Провести One Hot Encoding категоріального атрибута.
</li>
</ul>


Імпорт потрібних бібліотек

In [89]:
import pandas as pd
import numpy as np
import timeit
from IPython.display import display

Завантаження датасету, та зчитування його

In [90]:
# Читання з урахуванням розділювача і пропусків
# Вказуємо шлях до файлу
file_path = "household_power_consumption.txt"

# Читаємо дані
df = pd.read_csv(file_path, sep=';', low_memory=False)
print(df.head())
print(df.info())
print(df.describe())

         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column      

Підрахунок порожніх клітинок

In [91]:
# Тепер можна замінювати '?' на np.nan
df.replace('?', np.nan, inplace=True)

# Конвертуємо числові колонки
numeric_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Перевіримо пропущені значення
print(df.isnull().sum())

Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Блок коду що дає змогу працювати з часовими даними.

In [92]:
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df.set_index('Datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

Перевірка часу виконання очищення даних.

In [83]:
def clean_data(df):
    df_clean = df.copy()
    df_clean.replace('?', np.nan, inplace=True)
    for col in numeric_cols:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    df_clean.fillna(df_clean.mean(), inplace=True)
    return df_clean

# Профілювання часу
execution_time = timeit.timeit(lambda: clean_data(df), number=10)
print(f"Середній час виконання: {execution_time/10:.5f} секунд")

Середній час виконання: 0.27334 секунд


Нвступний блокі коду, де представлені функції що формують вибірки.

Вибірка за Global_active_power > 5 кВт

In [ ]:
def filter_active_power(df, threshold=5):
    """Обрати записи, де Global_active_power перевищує threshold кВт"""
    return df[df['Global_active_power'] > threshold]
df_filtered = filter_active_power(df)
df_filtered.head(8)
#print(df_filtered)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:35:00,5.412,0.470,232.78,23.2,0.0,1.0,17.0
2006-12-16 17:36:00,5.224,0.478,232.99,22.4,0.0,1.0,16.0
2006-12-16 17:37:00,5.268,0.398,232.91,22.6,0.0,2.0,17.0
2006-12-16 17:44:00,5.894,0.000,232.69,25.4,0.0,0.0,16.0
2006-12-16 17:45:00,7.706,0.000,230.98,33.2,0.0,0.0,17.0


Вибірка за струмом 19-20 А та порівняння груп споживання. 
Група 1 (пральна машина + холодильник) = Sub_metering_1 + Sub_metering_2
Група 2 (бойлер + кондиціонер) = Sub_metering_3 + Global_reactive_power (приклад, оскільки кондиціонера в датасеті немає)

In [71]:
def filter_current_and_compare(df, current_min=19, current_max=20):
    subset = df[(df['Global_intensity'] >= current_min) & (df['Global_intensity'] <= current_max)].copy()
    subset['group1'] = subset['Sub_metering_1'] + subset['Sub_metering_2']
    subset['group2'] = subset['Sub_metering_3'] + subset['Global_reactive_power']
    return subset[subset['group1'] > subset['group2']]
df_current = filter_current_and_compare(df)
df_current.head(10)
#print(df_current)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour,group1,group2
Datetime,,,,,,,,,,
2006-12-16 18:09:00,4.464,0.136,234.66,19.0,0.0,37.0,16.0,18,37.0,16.136
2006-12-17 01:04:00,4.582,0.258,238.08,19.6,0.0,13.0,0.0,1,13.0,0.258
2006-12-17 01:08:00,4.618,0.104,239.61,19.6,0.0,27.0,0.0,1,27.0,0.104
2006-12-17 01:19:00,4.636,0.140,237.37,19.4,0.0,36.0,0.0,1,36.0,0.140
2006-12-17 01:20:00,4.634,0.152,237.17,19.4,0.0,35.0,0.0,1,35.0,0.152
2006-12-17 01:21:00,4.652,0.142,237.92,19.4,0.0,36.0,0.0,1,36.0,0.142
2006-12-17 01:52:00,4.622,0.240,239.59,19.2,0.0,37.0,0.0,1,37.0,0.240
2006-12-17 09:08:00,4.762,0.088,237.44,20.0,0.0,38.0,0.0,9,38.0,0.088
2006-12-17 09:09:00,4.506,0.088,237.25,19.0,0.0,38.0,0.0,9,38.0,0.088


Випадкова вибірка 500000 записів та середні значення груп

In [68]:
def random_sample_means(df, n=500000):
    sample = df.sample(n=n, replace=False, random_state=42)
    sample['group1'] = sample['Sub_metering_1']
    sample['group2'] = sample['Sub_metering_2']
    sample['group3'] = sample['Sub_metering_3']
    means = sample[['group1','group2','group3']].mean()
    return sample, means
sample, means = random_sample_means(df)
print(sample,means)

                     Global_active_power  Global_reactive_power  Voltage  \
Datetime                                                                   
2010-07-07 18:10:00                0.256                  0.106   242.00   
2007-05-14 06:50:00                0.466                  0.352   237.22   
2007-09-26 18:10:00                0.758                  0.194   238.66   
2007-06-19 07:30:00                1.290                  0.046   240.64   
2010-05-10 04:43:00                0.428                  0.202   242.23   
...                                  ...                    ...      ...   
2007-04-26 21:58:00                1.892                  0.174   232.74   
2007-11-11 18:00:00                1.098                  0.050   241.16   
2010-05-17 23:04:00                0.306                  0.000   246.83   
2008-08-12 18:35:00                0.252                  0.232   242.05   
2009-02-17 02:33:00                0.288                  0.052   245.05   

           

Вибірка після 18:00 з основним споживанням на групі 2 та підбір кожного N-го

In [64]:
def evening_selection(df):
    # створюємо колонку з часом
    df['hour'] = df.index.hour
    subset = df[(df['hour'] >= 18) & (df['Global_active_power'] > 6)]
    
    # групи споживання
    subset['group1'] = subset['Sub_metering_1']  # пральна машина
    subset['group2'] = subset['Sub_metering_2']  # холодильник/освітлення
    subset['group3'] = subset['Sub_metering_3']  # інші

    # вибираємо ті, де група2 найбільша
    subset = subset[(subset['group2'] > subset['group1']) & (subset['group2'] > subset['group3'])]

    # ділимо на дві половини
    half = len(subset)//2
    first_half = subset.iloc[:half:3]   # кожен третій
    second_half = subset.iloc[half::4]  # кожен четвертий

    return pd.concat([first_half, second_half])
evening_df = evening_selection(df)
evening_df.head()
#print(evening_df)

C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2756428967.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['group1'] = subset['Sub_metering_1']  # пральна машина
C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2756428967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['group2'] = subset['Sub_metering_2']  # холодильник/освітлення
C:\Users\Ivan Moroz\AppData\Local\Temp\ipykernel_9352\2756428967.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour,group1,group2,group3
Datetime,,,,,,,,,,,
2006-12-16 18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0,18,0.0,37.0,17.0
2006-12-16 18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0,18,0.0,36.0,17.0
2006-12-28 20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0,20,1.0,36.0,17.0
2006-12-28 21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0,21,1.0,72.0,17.0
2006-12-28 21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0,21,1.0,73.0,17.0


Нормалізація та стандартизація

In [59]:
def normalize_and_standardize(df, cols):
    # Нормалізація [0,1]
    for col in cols:
        df[col+'_norm'] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    # Стандартизація (z-score)
    for col in cols:
        df[col+'_std'] = (df[col] - df[col].mean()) / df[col].std()
    
    return df
evening_df = normalize_and_standardize(evening_df, ['group1','group2','group3'])
evening_df.head()
#print(evening_df)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour,group1,group2,group3,group1_norm,group2_norm,group3_norm,group1_std,group2_std,group3_std
Datetime,,,,,,,,,,,,,,,,,
2006-12-16 18:05:00,6.052,0.192,232.93,26.2,0.0,37.0,17.0,18,0.0,37.0,17.0,0.000000,0.301887,0.607143,-0.520625,-0.964122,0.38096
2006-12-16 18:08:00,6.308,0.116,232.25,27.0,0.0,36.0,17.0,18,0.0,36.0,17.0,0.000000,0.283019,0.607143,-0.520625,-1.021632,0.38096
2006-12-28 20:58:00,6.386,0.374,236.63,27.0,1.0,36.0,17.0,20,1.0,36.0,17.0,0.023256,0.283019,0.607143,-0.437902,-1.021632,0.38096
2006-12-28 21:02:00,8.088,0.262,235.50,34.4,1.0,72.0,17.0,21,1.0,72.0,17.0,0.023256,0.962264,0.607143,-0.437902,1.048717,0.38096
2006-12-28 21:05:00,7.230,0.152,235.22,30.6,1.0,73.0,17.0,21,1.0,73.0,17.0,0.023256,0.981132,0.607143,-0.437902,1.106227,0.38096


Підрахунок коефіцієнтів Пірсона та Спірмена

In [ ]:
def correlation(df, col1, col2):
    # Пірсон 
    pearson = df[col1].corr(df[col2], method='pearson')
    
    # Спірмен (через ранги)
    rank1 = df[col1].rank()
    rank2 = df[col2].rank()
    spearman = rank1.corr(rank2)  # кореляція Пірсона між рангами
    return pearson, spearman
pearson, spearman = correlation(df, 'Global_active_power', 'Voltage')
print(pearson,spearman)

(np.float64(-0.3997616096289585), np.float64(-0.32521294259808614))

One Hot Encoding категоріального атрибута

In [62]:
def one_hot_encode(df, col):
    return pd.get_dummies(df, columns=[col], prefix=[col])
df_encoded = pd.get_dummies(df, columns=['hour'], prefix='hour')
df_encoded.head()
#print(df_encoded.head())

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,hour_0,hour_1,hour_2,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
Datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,False,False,False,...,False,False,False,True,False,False,False,False,False,False
